# Проект "Кто следущий?"

## Описание проекта

Оператор связи «Ниединогоразрыва.ком» хочет научиться прогнозировать отток клиентов. Если выяснится, что пользователь планирует уйти, ему будут предложены промокоды и специальные условия. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и договорах.

### Описание услуг

Оператор предоставляет два основных типа услуг: 

1. Стационарную телефонную связь. Возможно подключение телефонного аппарата к нескольким линиям одновременно.
2. Интернет. Подключение может быть двух типов: через телефонную линию (DSL*,* от англ. *digital subscriber line*, «цифровая абонентская линия») или оптоволоконный кабель (*Fiber optic*).  

Также доступны такие услуги:

- Интернет-безопасность: антивирус (*DeviceProtection*) и блокировка небезопасных сайтов (*OnlineSecurity*);
- Выделенная линия технической поддержки (*TechSupport*);
- Облачное хранилище файлов для резервного копирования данных (*OnlineBackup*);
- Стриминговое телевидение (*StreamingTV*) и каталог фильмов (*StreamingMovies*).

За услуги клиенты могут платить каждый месяц или заключить договор на 1–2 года. Доступны различные способы расчёта и возможность получения электронного чека.

### Описание данных

Данные состоят из файлов, полученных из разных источников:

- `contract_new.csv` — информация о договоре;
- `personal_new.csv` — персональные данные клиента;
- `internet_new.csv` — информация об интернет-услугах;
- `phone_new.csv` — информация об услугах телефонии.

Во всех файлах столбец `customerID` содержит код клиента.

Информация о договорах актуальна на 1 февраля 2020.

### Цель проекта

Создать модель, позволяющую на основе данных, предоставленных провайдером выявлять клиентов, которые потенциально могут расторгнуть договор обслуживания.

In [1]:
pip install skimpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta,date
from scipy import stats as st
from skimpy import clean_columns

## Загрузка и обзор данных

Данные будут загружены в следующие датафреймы:<br>

**dc** - contract_new.csv — информация о договоре;<br>
**dp** - personal_new.csv — персональные данные клиента;<br>
**di** - internet_new.csv — информация об интернет-услугах;<br>
**dph** - phone_new.csv — информация об услугах телефонии.

In [3]:
try:    
    dc = pd.read_csv('/datasets/contract_new.csv')
    dp = pd.read_csv('/datasets/personal_new.csv')
    di =  pd.read_csv('/datasets/internet_new.csv')
    dph =  pd.read_csv('/datasets/phone_new.csv')
except:
    dc = pd.read_csv('https://code.s3.yandex.net/datasets/contract_new.csv')
    dp = pd.read_csv('https://code.s3.yandex.net/datasets/personal_new.csv')
    di =  pd.read_csv('https://code.s3.yandex.net/datasets/internet_new.csv')
    dph =  pd.read_csv('https://code.s3.yandex.net/datasets/phone_new.csv')

#### Сбор информации о датафреймах

In [4]:
dc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


In [5]:
dp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [6]:
di.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


In [7]:
dph.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


In [8]:
#example = "ThisIsAGoodExample"
#print (re.sub( '(?<!^)(?=[A-Z])', '_', example ).lower())

In [9]:
#dc = clean_columns(dc)

In [10]:
dc.head()

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,2071.84
2,3668-QPYBK,2019-10-01,No,Month-to-month,Yes,Mailed check,53.85,226.17
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1960.6
4,9237-HQITU,2019-09-01,No,Month-to-month,Yes,Electronic check,70.70,353.5


In [11]:
dp.head()

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


In [12]:
di.head()

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


In [13]:
dph.head()

,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


#### Проверка на дубликаты

Во всех полях, кроме **customerID** сведения могут совпадать, поэтому проверке на дубликаты подвергается только **customerID**

In [14]:
dc.duplicated(['customerID']).sum()

0

In [15]:
dp.duplicated(['customerID']).sum()

0

In [16]:
di.duplicated(['customerID']).sum()

0

In [17]:
dph.duplicated(['customerID']).sum()

0

Проверка поля TotalCharges

In [18]:
dc[dc['TotalCharges'].str.contains(" ")]

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
488,4472-LVYGI,2020-02-01,No,Two year,Yes,Bank transfer (automatic),52.55,
753,3115-CZMZD,2020-02-01,No,Two year,No,Mailed check,20.25,
936,5709-LVOEQ,2020-02-01,No,Two year,No,Mailed check,80.85,
1082,4367-NUYAO,2020-02-01,No,Two year,No,Mailed check,25.75,
1340,1371-DWPAZ,2020-02-01,No,Two year,No,Credit card (automatic),56.05,
3331,7644-OMVMY,2020-02-01,No,Two year,No,Mailed check,19.85,
3826,3213-VVOLG,2020-02-01,No,Two year,No,Mailed check,25.35,
4380,2520-SGTTA,2020-02-01,No,Two year,No,Mailed check,20.00,
5218,2923-ARZLG,2020-02-01,No,One year,Yes,Mailed check,19.70,
6670,4075-WKNIU,2020-02-01,No,Two year,No,Mailed check,73.35,


Сведения отсутствуют в связи с отсуствием оплаты по договору, который был заключен в день предоставления данных заказчиком (01 февраля 2020 года). Эти данные можно удалить в связи с их ничтожным влиянием на общую картину

##### Вывод (обзор данных)

Исходные датафреймы пропусков и дубликатов не имеют.<br>
Таблицы dc и dp имеют одинаковое количество записей, поэтому, возможно, в них содержатся сведения об одних и тех же абонентах. Точнее можно будет сказать при объединении таблиц.<br>
В таблице dc поле **BeginDate** требуется привести к типу **data**. **TotalCharges** необходимо привести к **float**, предварительно удалив строки, содержащие пробелы (договор только что заключен и не оплачен)<br>
По всей видимости, данные экспортировались неправильно из какой-либо базы данных.<br>
Наименования столбцов необходимо привести к нижнему регистру с нижним подчеркиванием.<br>


В качестве целевого признака необходимо будет создать отдельный бинарный столбец на основе **EndDate**, в котором 1 - наличие договора о расторжении, 0 - в противном случае.<br>
Необходимо создать поле c количеством дней, прошедших с момента заключения договора, до даты 1 февраля 2020. На его основе, в частности можно будет проверить аномальные сроки действия договоров<br>

Для дальнейшего анализа данных и последущего моделирования необходимо будет исходные датафреймы объединять по **customerID**<br>

После слияния и заполнения пропусков бинарные значения Yes и No заменить на 0, 1

### План работы

1. Предобработка данных и конструирование новых признаков <br>

Выполнить обработку имен столбцов, работа с типами данных, удаление дубликатов, создание новых столбцов, слияние таблиц (merge), заполнение образовавшихся пропусков

2. Исследовательский анализ данных (EDA)<br>

Выявление закономерностей функционирования системы данных, которые, возможно, позволят выявить существенные признаки, потенциально оказывающих влияние на построение модели (построение "портрета" клиента, баланс классов, развитие структуры в динамике, построение визуализаций)

3. Исследование мультиколлинеарности<br>

Выявление зависимости между признаками, коллинеарность признаков с целевым признаком<br>
На основе этого отбираем признаки для модели (future selection), можно дополнитель но применить F-test



4. Разделение данных на выборки<br>

Так как будет использоваться GridSearch, то данные будут разбиты на **тренировочную** и **тестовую** выборки в отношении 3:1<br>
Кодирование признаков (OHE)<br>
Масштабирование признаков



5. Обучение моделей с подбором параметров 

LogisticRegression<br>
RandomForestClassifier<br>

для подбора параметра использовать GridSearchCV



6. Выбор лучшей модели по кросс-валидационной выборке

В качестве метрики используется ROC-AUC

7. Тестирование модели<br>

Выполнить тестирование модели на тестовой выборке<br>
проверить на адекватность (можно по ROC - кривой)



8. Выявление важности признаков (future importance)

Для дальнейшего отслеживания жизненного цикла модели<br>
Для внесения изменения в существующую модель в части исключения маловажных признаков из модели

9. Оценка точности модели, построение матрицы ошибок

10. Создание отчета

### Вопросы к заказчику:

1. Имеется ли сведения о времени выхода абонента в сеть(если это интернет), длительности пребывания, средней скорости на канале 
(Возможно покинувший недоволен скоростью, а пользуется в часы пиковой нагрузки)<br>
2. Какие скидки, улучшения и прочее провайдер в принципе может предложить клиентам (в зависимости от этого может изменяться анализ данных и сами предоставляемые данные)<br>
3. Кто будет читать отчет и насколько детализировать полученные результаты, кто будет принимать управленческие решения (от этого могут зависеть представляемые выводы, визуализация, представление данных)<br>

## Предобработка данных

#### Преобразование имен столбцов

Использование модуля **skimpy**

In [19]:
dc = clean_columns(dc)

8 column names have been cleaned

In [20]:
dp = clean_columns(dp)

4 column names have been cleaned

In [21]:
di = clean_columns(di)

8 column names have been cleaned

In [22]:
dph = clean_columns(dph)

2 column names have been cleaned

In [23]:
dc.columns.tolist()

['customer_id',
 'begin_date',
 'end_date',
 'type',
 'paperless_billing',
 'payment_method',
 'monthly_charges',
 'total_charges']

In [24]:
dp.columns.tolist()

['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents']

In [25]:
di.columns.tolist()

['customer_id',
 'internet_service',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies']

In [26]:
dph.columns.tolist()

['customer_id', 'multiple_lines']

#### Создание новых данных

Создание целевого столбца

In [27]:
dc['target'] = 0

Если в поле **end_date** находится значение, отличное от **No**, то в поле *target* устанавливается **1**

In [28]:
dc.loc[dc['end_date']!='No','target'] = 1

Преобразование типов

In [29]:
dc['begin_date'] = pd.to_datetime(dc['begin_date'])

In [30]:
dc['begin_date'].dtype

dtype('<M8[ns]')

In [31]:
dc.shape[0]

7043

In [32]:
#df = df.loc[df['STP'] != 1005092]
dc = dc.loc[~dc['total_charges'].str.contains(" ")]

In [33]:
dc.shape[0]

7032

In [34]:
dc['total_charges'] = dc['total_charges'].astype(float)

In [35]:
dc['total_charges'].dtype

dtype('float64')

**Расчет дней действия договора (contract_lifetime)**

In [36]:
def calc_days_between_date(row):
    '''
    Модуль для вычисления количества дней между двумя датами
    '''
    
    someday = date(2020,2,1)
    if row['end_date'] == 'No':
        return (someday - row['begin_date'].date()).days
    else:
        return (datetime.strptime(row['end_date'], '%Y-%m-%d').date() - row['begin_date'].date()).days


In [37]:
dc['contract_lifetime'] = dc.apply(calc_days_between_date, axis=1)

In [38]:
dc.head(10)

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,target,contract_lifetime
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04,0,31
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,2071.84,0,1036
2,3668-QPYBK,2019-10-01,No,Month-to-month,Yes,Mailed check,53.85,226.17,0,123
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1960.60,0,1371
4,9237-HQITU,2019-09-01,No,Month-to-month,Yes,Electronic check,70.70,353.50,0,153
5,9305-CDSKC,2019-03-01,No,Month-to-month,Yes,Electronic check,99.65,1150.96,0,337
6,1452-KIOVK,2018-04-01,No,Month-to-month,Yes,Credit card (automatic),89.10,2058.21,0,671
7,6713-OKOMC,2019-04-01,No,Month-to-month,No,Mailed check,29.75,300.48,0,306
8,7892-POOKP,2017-07-01,No,Month-to-month,Yes,Electronic check,104.80,3573.68,0,945
9,6388-TABGU,2014-12-01,2017-05-01,One year,No,Bank transfer (automatic),56.15,1628.35,1,882


Числовые значения присутствуют только в dc, поэтому описание данных извлекается из нее

In [39]:
dc.describe()

,monthly_charges,total_charges,target,contract_lifetime
count,7032.000000,7032.000000,7032.000000,7032.000000
mean,64.798208,2118.621822,0.156570,899.961320
std,30.085974,2112.736199,0.363421,682.738777
min,18.250000,19.050000,0.000000,28.000000
25%,35.587500,439.745000,0.000000,276.000000
50%,70.350000,1345.275000,0.000000,761.000000
75%,89.862500,3239.317500,0.000000,1461.000000
max,118.750000,9221.380000,1.000000,2314.000000


Аномальных значений в описании данных не наблюдается